In [ ]:
import json
import psycopg2
from psycopg2.extras import RealDictCursor
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

load_dotenv()

In [ ]:
# Fetch the values from environment variables
api_key = os.getenv("API_KEY")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

conn = psycopg2.connect(
    dbname="box_scores",
    user=db_user,
    password=db_pass,
    host=db_host,
    port=db_port
)

In [ ]:
query = """
WITH current_games AS (
    SELECT array_agg(game_id) AS game_ids
    FROM game
    WHERE game.date < CURRENT_DATE
)
SELECT pg.player_id, g.date, p.first_name, p.last_name, pg.fgm, pg.fga, pg.fg_pct, pg.fg3m, pg.fg3a, pg.fg3_pct, pg.ftm, pg.fta, pg.ft_pct, pg.oreb, pg.dreb, pg.reb, pg.ast, pg.stl, pg.blk, pg.turnover, pg.pf, pg.pts
FROM player_game pg
INNER JOIN player p ON pg.player_id=p.player_id
INNER JOIN game g ON pg.game_id=g.game_id
WHERE pg.game_id = ANY (SELECT unnest(game_ids) FROM current_games);
"""

In [ ]:
df = pd.read_sql_query(query, conn)
conn.close()
df.to_json('../data/preprocessed.json')

In [ ]:
df.head()

In [ ]:
with open('../data/prizepicks.json') as f:
    data=json.load(f)

projections = data['data']
records = []
for projection in projections:
    attributes = projection.get('attributes', {})
    relationships = projection.get('relationships', {})
    new_player_data = relationships.get('new_player', {}).get('data', {})
    player_id = new_player_data.get('id', None)

    # Extract `display_name` from `new_player` attributes (if available)
    display_name = None
    if new_player_data and 'attributes' in new_player_data:
        display_name = new_player_data['attributes'].get('display_name', "Unknown")

    # Record data
    record = {
        'id': projection.get('id'),
        'type': projection.get('type'),
        'description': attributes.get('description'),
        'line_score': attributes.get('line_score'),
        'stat_type': attributes.get('stat_type'),
        'start_time': attributes.get('start_time'),
        'status': attributes.get('status'),
        'game_id': attributes.get('game_id'),
        'player_id': player_id,
    }
    records.append(record)

df = pd.DataFrame(records)
df.head()

In [ ]:
player_data = []
for row in data['included']:
    if row['type'] == 'new_player': 
        player_id = row['id']
        attributes = row.get('attributes', {})
        display_name = attributes.get('display_name', 'Unknown')
        player_data.append({'id': player_id, 'name': display_name})

player_data_df = pd.DataFrame(player_data)
player_data_df.head()

In [ ]:
df_cleaned = df.merge(player_data_df, left_on='player_id', right_on='id', how='left').drop(columns=['id_x', 'game_id', 'player_id', 'id_y'])
df_cleaned.head()

In [ ]:
df_cleaned.to_json('../data/prizepicks_cleaned.json')